In [ ]:
import datetime
import geopandas as gpd
import itertools
import numpy as np
import pandas as pd
import xarray as xr

import datacube

import sys

sys.path.insert(1, "../Tools/")
import dea_tools.bandindices
import dea_tools.datahandling
from dea_tools.spatial import xr_rasterize
from dea_tools.dask import create_local_dask_cluster
import dea_tools.wetlands

# Create local dask cluster to improve data load time
client = create_local_dask_cluster(return_client=True)

dc = datacube.Datacube(app="DEA_Wetlands_Insight_Tool")

In [2]:
# some things I might change
time = ("1985-01-01", "2024-01-31")
# time = ('1985-01-01', '2024-01-01')

# Define which spectral bands are being used in the analysis
bands = [
    f"nbart_{band}" for band in ("blue", "green", "red", "nir", "swir_1", "swir_2")#, "panchromatic")
]

In [5]:
poly = gpd.read_file("../Supplementary_data/DEA_Wetlands_Insight_Tool/wetmap_plots_Buffer_100_3577.shp")
poly.geometry[0] #
poly

,gid,plot_id,nw_latitud,nw_longitu,nw_bearing,centroid_l,centroid_1,originalpr,localityno,surveyed_2,hydrology_,BUFF_DIST,ORIG_FID,SHAPE_Leng,SHAPE_Area,geometry
0,1,Black_DC_01,-36.141010,145.451500,0.000000,-36.141055,145.451555,GBCMA Damien Cook,NaN,Y,Y,100.0,1,0.006739,0.000004,"POLYGON ((1208951.490 -4012686.516, 1208941.54..."
1,2,Black_DC_02,-36.141010,145.451900,0.000000,-36.141055,145.451956,GBCMA Damien Cook,NaN,Y,Y,100.0,2,0.006739,0.000004,"POLYGON ((1208987.313 -4012690.301, 1208977.36..."
2,3,Black_DC_03,-36.141110,145.452600,0.000000,-36.141155,145.452656,GBCMA Damien Cook,NaN,Y,Y,100.0,3,0.006739,0.000004,"POLYGON ((1209048.824 -4012707.971, 1209038.87..."
3,4,Black_DC_04,-36.141210,145.453100,0.000000,-36.141255,145.453156,GBCMA Damien Cook,NaN,Y,Y,100.0,4,0.006739,0.000004,"POLYGON ((1209092.449 -4012723.723, 1209082.50..."
4,5,BLSW-01-End,-36.142115,145.452233,30.948000,-36.142177,145.452252,WetMAP Stage3,NaN,N,N,100.0,5,0.006741,0.000004,"POLYGON ((1209006.065 -4012818.903, 1208997.35..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
409,409,WOSW-06-End,-36.163873,143.719556,36.903000,-36.163936,143.719567,WetMAP Stage3,NaN,N,N,100.0,410,0.006743,0.000004,"POLYGON ((1053531.951 -3999877.580, 1053523.23..."
410,410,WOSW-06-Start,-36.164201,143.719322,36.902663,-36.164264,143.719333,WetMAP Stage3,NaN,TBS,Y,100.0,411,0.006743,0.000004,"POLYGON ((1053507.672 -3999911.878, 1053498.95..."
411,120,WT12,-38.136900,147.179000,0.000000,-38.136945,147.179057,WGCMA Frood et al,Lower Latrobe wetlands complex,Y,Y,100.0,412,0.006842,0.000004,"POLYGON ((1337362.058 -4249200.661, 1337352.05..."
412,121,WT13,-38.136700,147.181000,0.000000,-38.136745,147.181057,WGCMA Frood et al,Lower Latrobe wetlands complex,Y,Y,100.0,413,0.006842,0.000004,"POLYGON ((1337540.081 -4249199.716, 1337530.07..."


In [6]:
for index, wetland in poly.iterrows():
    index

In [ ]:
# This cell loads and plots the wetlands polygon. If no output, check your polygon
poly = gpd.read_file("../Supplementary_data/DEA_Wetlands_Insight_Tool/wetmap_plots_Buffer_100_3577.shp")
for index, wetland in poly.iterrows():
    # get wetland point geodata
    # Specifying coordinate reference system of the polygon.
    gpgon = datacube.utils.geometry.Geometry(wetland.geometry, crs=poly.crs)
    
    # Load Landsat 5, 7 and 8 data. Not including Landsat 7 SLC off period (31-05-2003 to 06-04-2022).
    ds = dea_tools.datahandling.load_ard(
        dc,
        products=["ga_ls8c_ard_3", "ga_ls7e_ard_3", "ga_ls5t_ard_3"],
        ls7_slc_off=False,
        measurements=bands,
        geopolygon=gpgon,
        output_crs="EPSG:3577",
        resolution=(-30, 30),
        resampling={"fmask": "nearest", "*": "bilinear"},
        time=time,
        #group_by="solar_day",
        dask_chunks={},
    )

    # Load into memory using Dask
    ds.load()    

ds_wo = dc.load(
    "ga_ls_wo_3", resampling="nearest", group_by="solar_day", like=ds, dask_chunks={}
)
ds_fc = dc.load(
    "ga_ls_fc_3", resampling="nearest", group_by="solar_day", like=ds, dask_chunks={}
)

# Load data into memory
ds_wo.load()
ds_fc.load()

#locate and remove any observations which arent in all three datsets 
missing = set()
for t1, t2 in itertools.product(
    [ds_fc.time.values, ds_wo.time.values, ds.time.values], repeat=2
):
    missing_ = set(t1) - set(t2)
    missing |= missing_

ds_fc = ds_fc.sel(time=[t for t in ds_fc.time.values if t not in missing])
ds = ds.sel(time=[t for t in ds.time.values if t not in missing])
ds_wo = ds_wo.sel(time=[t for t in ds_wo.time.values if t not in missing])

#calculate tasseled cap wetness from landsat data 
tcw = dea_tools.bandindices.calculate_indices(
    ds, index="TCW", collection="ga_ls_3", normalise=False, drop=True, inplace=False
)

#divide fracitonal cover by 100 to keep them in (0,1). keeps data types the same in he output raster
bs = ds_fc.bs / 100
pv = ds_fc.pv / 100
npv = ds_fc.npv / 100

#generate the WIT raster bands, create an emty datset called ouput_rast and population with values from input datsets 
rast_names = ["pv", "npv", "bs", "wet", "water"]
output_rast = {n: xr.zeros_like(bs) for n in rast_names}

output_rast["bs"].values[:] = bs
output_rast["pv"].values[:] = pv
output_rast["npv"].values[:] = npv

#Threshold TCW at -350, with values above this threshold used to characterise 'wet' pixels 
# Rasterise the shapefile where poly is the vector data and pv is the xarray template
poly_raster = xr_rasterize(poly, pv) > 0

# Mask includes No data, Non contiguous data, Cloud shadow, Cloud, and water.
# See https://docs.dea.ga.gov.au/notebooks/DEA_products/DEA_Water_Observations.html#Understanding-WOs-bit-flags for more detail.
mask = (ds_wo.water & 0b01100011) == 0
mask &= poly_raster

# Set open water to water present and classified as water as per Water Observations and bit flags
open_water = ds_wo.water & (1 << 7) > 0

# Set wet pixels where not masked and above threshold of -350
wet = tcw.where(mask).TCW > -350

# Adding wet and water values to output raster

# TCW
output_rast["wet"].values[:] = wet.values.astype(float)
for name in rast_names[:3]:
    output_rast[name].values[wet.values] = 0

# WO
output_rast["water"].values[:] = open_water.values.astype(float)
for name in rast_names[:4]:
    output_rast[name].values[open_water.values] = 0
    
# Masking again
ds_wit = xr.Dataset(output_rast).where(mask)

# Calculate percentage missing
pc_missing = (~mask).where(poly_raster).mean(dim=["x", "y"])

ds_wit = ds_wit.where(pc_missing < 0.1)


#normalise fractional cover values in WIT result 
# Convert ds_wit: XArray.Dataset to polygon_base_df: pandas.DataFrame

polygon_base_df = pd.DataFrame()
polygon_base_df["date"] = ds_wit.time.values

for band in rast_names:
    polygon_base_df[band] = ds_wit[band].mean(dim=["x", "y"])

polygon_base_df = dea_tools.wetlands.normalise_wit(polygon_base_df)

Finding datasets
    ga_ls8c_ard_3
    ga_ls7e_ard_3 (ignoring SLC-off observations)
    ga_ls5t_ard_3
Applying fmask pixel quality/cloud mask
Returning 1253 time steps as a dask array


/env/lib/python3.10/site-packages/rasterio/warp.py:344: NotGeoreferencedWarning: Dataset has no geotransform, gcps, or rpcs. The identity matrix will be returned.
  _reproject(


Finding datasets
    ga_ls8c_ard_3
    ga_ls7e_ard_3 (ignoring SLC-off observations)
    ga_ls5t_ard_3
Applying fmask pixel quality/cloud mask
Returning 1253 time steps as a dask array
Finding datasets
    ga_ls8c_ard_3
    ga_ls7e_ard_3 (ignoring SLC-off observations)
    ga_ls5t_ard_3
Applying fmask pixel quality/cloud mask
Returning 1253 time steps as a dask array
Finding datasets
    ga_ls8c_ard_3
    ga_ls7e_ard_3 (ignoring SLC-off observations)
    ga_ls5t_ard_3
Applying fmask pixel quality/cloud mask
Returning 1253 time steps as a dask array
Finding datasets
    ga_ls8c_ard_3
    ga_ls7e_ard_3 (ignoring SLC-off observations)
    ga_ls5t_ard_3
Applying fmask pixel quality/cloud mask
Returning 1253 time steps as a dask array


2024-02-19 05:09:22,123 - distributed.utils_perf - WARNING - full garbage collections took 11% CPU time recently (threshold: 10%)
2024-02-19 05:11:45,088 - distributed.utils_perf - WARNING - full garbage collections took 11% CPU time recently (threshold: 10%)
2024-02-19 05:14:26,005 - distributed.utils_perf - WARNING - full garbage collections took 11% CPU time recently (threshold: 10%)
2024-02-19 05:17:33,263 - distributed.utils_perf - WARNING - full garbage collections took 11% CPU time recently (threshold: 10%)


Finding datasets
    ga_ls8c_ard_3
    ga_ls7e_ard_3 (ignoring SLC-off observations)
    ga_ls5t_ard_3
Applying fmask pixel quality/cloud mask
Returning 1253 time steps as a dask array


...

2024-02-19 06:55:37,492 - distributed.utils_perf - WARNING - full garbage collections took 14% CPU time recently (threshold: 10%)
2024-02-19 06:57:33,453 - distributed.utils_perf - WARNING - full garbage collections took 14% CPU time recently (threshold: 10%)
2024-02-19 06:59:55,473 - distributed.utils_perf - WARNING - full garbage collections took 14% CPU time recently (threshold: 10%)
2024-02-19 07:02:57,512 - distributed.utils_perf - WARNING - full garbage collections took 14% CPU time recently (threshold: 10%)


Finding datasets
    ga_ls8c_ard_3
    ga_ls7e_ard_3 (ignoring SLC-off observations)
    ga_ls5t_ard_3
Applying fmask pixel quality/cloud mask
Returning 1253 time steps as a dask array


2024-02-19 07:06:31,951 - distributed.utils_perf - WARNING - full garbage collections took 14% CPU time recently (threshold: 10%)
2024-02-19 07:08:16,553 - distributed.utils_perf - WARNING - full garbage collections took 14% CPU time recently (threshold: 10%)
2024-02-19 07:10:14,360 - distributed.utils_perf - WARNING - full garbage collections took 14% CPU time recently (threshold: 10%)
2024-02-19 07:12:35,241 - distributed.utils_perf - WARNING - full garbage collections took 14% CPU time recently (threshold: 10%)
2024-02-19 07:15:24,032 - distributed.utils_perf - WARNING - full garbage collections took 14% CPU time recently (threshold: 10%)


Finding datasets
    ga_ls8c_ard_3
    ga_ls7e_ard_3 (ignoring SLC-off observations)
    ga_ls5t_ard_3
Applying fmask pixel quality/cloud mask
Returning 1253 time steps as a dask array


2024-02-19 07:18:42,199 - distributed.utils_perf - WARNING - full garbage collections took 13% CPU time recently (threshold: 10%)


In [ ]:
#create WIT Plot 
polygon_name = wetland.plot_id
png_name = polygon_name  # file will be png_name.png

In [ ]:
dea_tools.wetlands.display_wit_stack_with_df(polygon_base_df, polygon_name, png_name)

In [ ]:
polygon_base_df.to_csv(wetland.plot_id + .csv)#wetland.plot_id + '.csv'